<a href="https://colab.research.google.com/github/rituparna50/MarsPosition/blob/main/Near_side_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using HMI near side images to validate MarsCam near side images.
We have already figured out a way to identify dates when the opening angle is between 90-270 degrees. But that is for far-side. In order to validate the utilisation of the MarsCam images, we first need to do near-side validation.

In [6]:
import astropy
import numpy as np

In [7]:
from astropy.time import Time
from astropy import units as u

In [8]:
DateTime = "2023-11-01 09:00:00"
when = Time(DateTime)
print(when)

2023-11-01 09:00:00.000


# Difference between Astropy and Skyfield libraries, even though both are Python packages

Astropy - Designed to handle a wide range of astronomical tasks from time and coordinate trasnformations to unit conversions, data visualization, cosmological calcualtions etc. Aims to provide a one-stop solution for many of the routine tasks in astronomy and astrophysics. Large and complex as it has a wide range of functionalities.

Skyfield - More focsed. Primary goal is to provide accurate and high-performance tools for computing the *positions of celestial bodies*. Much more high precision in terms of speed when computing positions using the JPL ephemerides.



In [ ]:
!pip install skyfield datetime
#pip install datetime

# Skyfield and datetime are two different python packages we will need.
# Could not write as
    # pip install skyfield
    # pip install datetime

  # Have to write as -- pip install skyfield datetime --> Together

In [17]:
from skyfield.api import load, Topos
  #Load and topos are being imported from the skyfield.api module
  # skyfield.api is a module within the skyfield astronomical library. The term API is Application Programming Interface",
  #which is a set of predefined classes, functions and methods

Load function is used to fetch and manage various astronomical data files, like the ephemerides from JPL. These files contain the data necessary for Skyfield to compute the positions of celestial bodies.

Topos Class - allows users to specifiy a location on the earth's surface using latitude and longitutde (sometimes also elevation). This is useful when you want to comute the psoition of celestial objects as seen from a specific place on earth.

**Overall, skyfield.api aims to simplify the user's interaction with the library by providing a set of high-level tools and interfaces. It abstracts away many of the intricacies and lets users perform common astronomical tasks with ease.**

# **HMI Image Lookup**

The Helioseismic and Magnetic Imager (HMI) on the Solar Dynamics Observatory (SDO) provides high resolution images of the Sun in different observble, including magentograms, dopplergrams and continuum intensity

In [ ]:
pip install sunpy

In [ ]:
pip install drms

In [21]:
pip install zeep

In [22]:
# Import installed libraries
import drms
import sunpy

DRMS stands for Data Record Management System. Provides a means to access solar data fromt the Joint Science Operations Center (JSOC). This is useful for researchers who want to access images captured by HMI, AIA

Zeep is a Python SOAP client. Simple Object Access Protocol is a messaging protocol that allows programs to communicate over the internet. Zeep helps you send SOAP requests and handle the responses in Python.

In [25]:
from sunpy.net import Fido, attrs as a

In [26]:
# Set up the connection to JSOC
c = drms.Client(email='ritu@sun.stanford.edu')

  #Thic code creates a connection to the JSOC using the drms library.
  # drms.Cient() creates an instance of the Client class provided by the drms library.
  #The "Client" class is designed to establish a connection to the JSOC database,
  # allowing me to subsequently query and request data from JSOC using the connection


In [27]:
# Mention user defined or desired date for which solar image is neeed
date = '2023-11-01 09:00'

# Construct the data series string for HMI near-side continuum images
series = 'hmi.Ic_45s'
time_string = f'{date}T00:00:00z/{date}T23:59:59Z'  #This willc over the entire day

In [28]:
#Semd the data export request to JSOC
jsoc_series = 'hmi.td_fsi_12h'
jsoc_email = 'ritu@sun.stanford.edu'
first_rec = '2023.05.01_00:00:00_TAI'
last_rec = '2023.05.31_12:00:00_TAI'



In [29]:
query = Fido.search(
    a.Time(first_rec, last_rec),
    a.jsoc.Series(jsoc_series),
    a.jsoc.Notify(jsoc_email)
)
print(query)

Results from 1 Provider:

62 Results from the JSOCClient:
Source: http://jsoc.stanford.edu

         T_REC          TELESCOP  INSTRUME  CAR_ROT
----------------------- -------- ---------- -------
2023.05.01_00:00:00_TAI  SDO/HMI HMI_FRONT2    2270
2023.05.01_12:00:00_TAI  SDO/HMI HMI_FRONT2    2270
2023.05.02_00:00:00_TAI  SDO/HMI HMI_FRONT2    2270
2023.05.02_12:00:00_TAI  SDO/HMI HMI_FRONT2    2270
2023.05.03_00:00:00_TAI  SDO/HMI HMI_FRONT2    2270
2023.05.03_12:00:00_TAI  SDO/HMI HMI_FRONT2    2270
2023.05.04_00:00:00_TAI  SDO/HMI HMI_FRONT2    2270
2023.05.04_12:00:00_TAI  SDO/HMI HMI_FRONT2    2270
2023.05.05_00:00:00_TAI  SDO/HMI HMI_FRONT2    2270
2023.05.05_12:00:00_TAI  SDO/HMI HMI_FRONT2    2270
                    ...      ...        ...     ...
2023.05.26_12:00:00_TAI  SDO/HMI HMI_FRONT2    2271
2023.05.27_00:00:00_TAI  SDO/HMI HMI_FRONT2    2271
2023.05.27_12:00:00_TAI  SDO/HMI HMI_FRONT2    2271
2023.05.28_00:00:00_TAI  SDO/HMI HMI_FRONT2    2271
2023.05.28_12:00:00_TAI 

In [30]:
import os
from astropy.io import fits

In [32]:
import matplotlib.pyplot as plt
import sunpy.map
from scipy import ndimage

In [33]:
from matplotlib import ticker